<a href="https://colab.research.google.com/github/gupta24789/Data_Science_Projects/blob/master/colab_notebooks/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install tqdm

### Downloading the Dogs vs Cats dataset

In [40]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2019-09-18 08:14:41--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  49.4MB/s    in 1.3s    

2019-09-18 08:14:42 (49.4 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



### Import project dependencies

In [42]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0-alpha0'

### Unzipping the Dogs vs Cats dataset

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()

### Seting up dataset paths

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

#### Loading the pre-trained model (MobileNetV2)

In [45]:
IMG_SHAPE = (128, 128, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
base_model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 64, 64, 32)   128         Conv1[0][0]                      
________________

#### Freezing the base model

In [0]:
base_model.trainable = False

#### Defining the custom head for our network

In [47]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [49]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
global_average_layer

<tf.Tensor 'global_average_pooling2d_1/Mean:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

#### Defining the model

In [51]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

#### Creating Data Generators

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [55]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


#### Training the model

In [56]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 50s 3s/step - loss: 0.2446 - accuracy: 0.8960 - val_loss: 0.1455 - val_accuracy: 0.9510
Epoch 2/5
16/16 [==============================] - 11s 711ms/step - loss: 0.0302 - accuracy: 0.9965 - val_loss: 0.1255 - val_accuracy: 0.9560
Epoch 3/5
16/16 [==============================] - 11s 715ms/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 0.1659 - val_accuracy: 0.9480
Epoch 4/5
16/16 [==============================] - 11s 710ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.1695 - val_accuracy: 0.9520
Epoch 5/5
16/16 [==============================] - 11s 718ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.2389 - val_accuracy: 0.9460


#### Transfer learning model evaluation

In [57]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9459999799728394


## Fine Tuning
There are a few pointers:

DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model.



In [58]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [0]:
base_model.trainable = True

In [0]:
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [62]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 28s 2s/step - loss: 8.5822e-04 - accuracy: 1.0000 - val_loss: 0.5661 - val_accuracy: 0.9110
Epoch 2/5
16/16 [==============================] - 8s 515ms/step - loss: 3.8058e-04 - accuracy: 1.0000 - val_loss: 0.7066 - val_accuracy: 0.9000
Epoch 3/5
16/16 [==============================] - 9s 535ms/step - loss: 1.5942e-04 - accuracy: 1.0000 - val_loss: 0.7536 - val_accuracy: 0.9030
Epoch 4/5
16/16 [==============================] - 8s 522ms/step - loss: 3.6526e-05 - accuracy: 1.0000 - val_loss: 0.7035 - val_accuracy: 0.9080
Epoch 5/5
16/16 [==============================] - 8s 526ms/step - loss: 5.0110e-05 - accuracy: 1.0000 - val_loss: 0.8270 - val_accuracy: 0.8950


In [63]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.8949999809265137
